---
title: "Problem Set 1"
author: "Daniel Avila"
date: "10-5-2024"
format: html
execute:
    eval: true
    echo: true
---


1. This submission is my work alone and complies with the 30538 integrity policy.” Add
your initials to indicate your agreement: **DA**

2. “I have uploaded the names of anyone I worked with on the problem set here” **DA**
(1 point)

3. Late coins used this pset: **1** Late coins left after submission: **3**

4. Knit your ps1.qmd to make ps1.pdf.
• The PDF should not be more than 25 pages. Use head() and re-size figures when
appropriate.

5. Push ps1.qmd and ps1.pdf to your github repo. It is fine to use Github Desktop.

6. Submit ps1.pdf via Gradescope (4 points)

7. Tag your submission in Gradescope

**Background**

Read this article and this shorter article. If you are curious to learn more, this page has all
of the articles that ProPublica has done on this topic. 

***PS1***

Read in one percent sample (15 Points)


In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import os
import time

os.chdir("C:/Users/danie/OneDrive/Documents/GitHub/ppha30538_fall2024/problem_sets/ps1/data")

df = pd.read_csv("parking_tickets_one_percent.csv")
df.head()

1. To help you get started, we pushed a file to the course repo called parking_tickets_one_percent.csv which gives you a one percent sample of tickets. We constructed the sample by selecting 1 ticket numbers that end in 01. How long does it take to read in this file? (Find a function to measure how long it takes the command to run. Note that everytime you run, there will be some difference in how long the code takes to run). Add an assert statement which verifies that there are 287458 rows. 


In [ ]:
start = time.time()

pd.read_csv("parking_tickets_one_percent.csv")

end = time.time()

print(end - start)

2. Using a function in the os library calculate how many megabytes is the CSV file? 
Using math, how large would you predict the full data set is?


In [ ]:
df_size = os.path.getsize("parking_tickets_one_percent.csv")
print(df_size / 1048576)

In [ ]:
print(df_size*100/1048576)

3. The rows on the dataset are ordered or sorted by a certain column by default. Which
column? Then, subset the dataset to the first 500 rows and write a function that tests
if the column is ordered.

The rows in the dataset are ordered by the "issue_date" column. 

In [ ]:
df_500 = df.loc[:499]
df_500.shape

df_datetime = df
df_datetime["issue_date"] = pd.to_datetime(df_datetime["issue_date"])

def check():
    for i in range(0, 500):
        if df_datetime.loc[i, "issue_date"] > df_datetime.loc[i+1, "issue_date"]:
            print("not ordered by issue date")

check()

**Cleaning the data and benchmarking (15 Points)**

1. How many tickets were issued in the data in 2017? How many tickets does that imply
were issued in the full data in 2017? How many tickets are issued each year according
to the ProPublica article? Do you think that there is a meaningful difference?

In [ ]:
print(f"there are {df.shape[0]} tickets in this 2017 dataset")

print(f"this implies that there are approximately {df.shape[0]*100} tickets throughout 2017")

print("however, there were \"greater than 3 million tickets used each year\" according to the ProPublica article.")

print("our resulting figures indicate that the ProPublica article is overestimating by nearly 130,000 tickets, or approximately .4%. However, note that their claim is for parking tickets, speeding ticktes, and other infractions, whereas our dataset is only for parking violations. Therefore I believe that the \"greater than 3 million\" claim is acceptable.")

2. Pooling the data across all years what are the top 20 most frequent violation types?
Make a bar graph to show the frequency of these ticket types. Format the graph such
that the violation descriptions are legible and no words are cut off.


In [ ]:
groupby = df.groupby("violation_description").count()
groupby = groupby.nlargest(20, "ticket_number")
groupby = groupby.reset_index()

alt.Chart(groupby).mark_bar().encode(
    alt.Y("violation_description", 
        axis = alt.Axis(labelFontSize = 7)),
    alt.X("ticket_number", 
        axis = alt.Axis(labelFontSize = 7))
)

#had inspiration from chatgpt about identifying the labelfontsize and axis variables in the y and x axis

**Visual Encoding (15 Points)**

1. In lecture 2, we discussed how Altair thinks about categorizing data series into four
different types. Which data type or types would you associate with each column in the
data frame? Your response should take the form of a markdown table where each row
corresponds to one of the variables in the parking tickets dataset, the first column is the
variable name and the second column is the variable type or types. If you argue that a
column might be associated with than one type, explain why in writing below the table.


In [ ]:
columns = pd.DataFrame(df.columns)
dtype_dict = {1:"nominal",
              2: "ordinal",
              3: "quantitative",
              4: "temporal"}

dtypes = [2, 2, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 2, 4, 2, 1, 1, 1, 1]

columns["dtypes"] = dtypes
columns["dtypes"] = columns["dtypes"].map(dtype_dict)

columns = columns.T
columns

2. Compute the fraction of time that tickets issued to each vehicle make are marked as paid.
Show the results as a bar graph. Why do you think that some vehicle makes are more
or less likely to have paid tickets?


In [ ]:
df_datetime["ticket_queue_date"] = pd.to_datetime(df_datetime["ticket_queue_date"])
paid = df_datetime[df_datetime["ticket_queue"] == "Paid"]
paid["difference"] = paid["ticket_queue_date"] - paid["issue_date"]
paid_gb = paid.groupby("vehicle_make")
dif_mean = paid_gb["difference"].mean().reset_index()
dif_mean["difference"] = dif_mean["difference"].apply(lambda x: x.total_seconds())

alt.Chart(dif_mean).mark_bar().encode(
    alt.X("difference"),
    alt.Y("vehicle_make")
)

Individuals who have older vehicles are more likely to have purchased that car second hand which can be an indicator for less disposable income for durables such as cars. If the individual has less disposable income, then theroetically they would also have less income to pay their tickets. So older cars are more likely to have more tickets, because tickets accumulate when they cannot be paid. 

Vehicles 

3. Make a plot for the number of tickets issued over time by adapting the Filled Step Chart
example online.

In [ ]:
#had to find a pd.Grouper because I had too many rows in my dataset without it.
#found via chatgpt

grouped_date = df.groupby(pd.Grouper(key="issue_date", freq="ME")).count().reset_index()

alt.Chart(grouped_date).mark_area(
    color="lightblue",
    interpolate='step-after',
    line=True
    ).encode(
    alt.X("issue_date"),
    alt.Y("vehicle_make")
)

4. Make a plot for the number of tickets issued by month and day by adapting the Annual
Weather Heatmap example online.


In [ ]:
alt.Chart(grouped_date).mark_rect().encode(
    x=alt.X("date(issue_date)", axis=alt.Axis(format="%e", labelAngle=0)),
    y=alt.Y("month(issue_date)", title="Month"),
    color=alt.Color("max(vehicle_make)", legend=alt.Legend(title=None)),
    tooltip=[
        alt.Tooltip("monthdate(date)", title="Date"),
        alt.Tooltip("max(issue_date)", title="Max Temp"),
    ],
).configure_view(step=13, strokeWidth=0).configure_axis(domain=False)

5. Subset to the five most common types of violations. Make a plot for the number of
tickets issued over time by adapting the Lasagna Plot example online.

In [ ]:
groupby = df.groupby("violation_description").count()
groupby = groupby.nlargest(20, "ticket_number")

alt.Chart(groupby, width=300, height=200).mark_rect().encode(
    x=alt.X("issue_date"),
    y=alt.Y("ticket_number:N"),
)

6. Compare and contrast the plots you made for the prior three questions. What are the
pros and cons of each plot?

I mean these are so far off from what I know you are looking for...but the bar graph is great for showing ordered list comparisons of quantitative data. The heatmap is great for creating something of a geographic effect for  quantitative data, to identify areas of frequency. And the lasagna graph incorporates a time element to that conversation.

7. Suppose that the lesson you want a reader to take away is that the enforcement of
violations is not evenly distributed over time? Which plot is best and why?
Yeah I wasnt able to get that from my plots. But I would think that the lasagna plot would show that best since it is acting as a heatmap, which would indicate the number of tickets per month per year. So that would give a good sense of temporal patterns. 